In [29]:
from urllib.request import Request, urlopen as uReq
from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
import io
import json
import os
import pandas as pd
from config import conn_host, conn_database, conn_user, conn_password
from IPython.display import clear_output

In [59]:
league = "Major-League-Soccer"
league_id = 22
single_year_season = True
start_season = 2018
end_season = 2024

In [60]:
option = Options()
# option.headless = True
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
# driver.maximize_window()

C:\Users\caiog\AppData\Local\Temp\ipykernel_28724\3468875481.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [61]:
def get_teams_squad_id(home_td_index, tds):
    if len(tds[home_td_index].find_element(By.TAG_NAME, 'a').get_attribute("href").split('/')) > 7:
        squad_id_index = -3
    else:
        squad_id_index = -2
    home_squad_id = tds[home_td_index].find_element(By.TAG_NAME, 'a').get_attribute("href").split('/')[squad_id_index]
    away_squad_id = tds[home_td_index+4].find_element(By.TAG_NAME, 'a').get_attribute("href").split('/')[squad_id_index]
    return home_squad_id, away_squad_id

seasons = dict()
seasons_squad_ids = dict()

for season in range(start_season, end_season):
    seasons[season] = []
    seasons_squad_ids[season] = []
    season_str = str(season) if single_year_season else f"{season}-{season+1}"
    
    url = f"https://fbref.com/en/comps/{league_id}/{season_str}/schedule/{league}-Scores-and-Fixtures"
    driver.get(url)

    driver.maximize_window()

    fb = driver.find_element(By.CLASS_NAME, 'fb')
    rows = fb.find_elements(By.XPATH, '//table/tbody/tr')

    total_games = 0
    for i, r in enumerate(rows):
        clear_output()
        print(f"{season}/{end_season-1} {i}/{len(rows)}")
        if not r.text: continue
        tds = r.find_elements(By.XPATH, './/child::td')
        week = r.find_element(By.XPATH, './/child::th').text
            
        if len(tds) == 12:
            date, _, home_team, home_xg, score, away_xg, away_team, _, _, _, _, _ = [t.text for t in tds]
            home_squad_id, away_squad_id = get_teams_squad_id(2, tds)
        elif len(tds) == 13:
            _, date, _, home_team, home_xg, score, away_xg, away_team, _, _, _, _, _ = [t.text for t in tds]
            home_squad_id, away_squad_id = get_teams_squad_id(3, tds)
        elif len(tds) == 14:
            _, _, date, _, home_team, home_xg, score, away_xg, away_team, _, _, _, _, _ = [t.text for t in tds]
            home_squad_id, away_squad_id = get_teams_squad_id(4, tds)
        else: continue
            
        if not score: continue
        seasons_squad_ids[season].extend([(home_squad_id, home_team), (away_squad_id, away_team)])
        home_score, away_score = score.split('–')
        match_info = [date, week, home_team, home_xg, home_score, away_score, away_xg, away_team]
        seasons[season].append(match_info)
        total_games += 1
    
    seasons_squad_ids[season] = set(seasons_squad_ids[season])

2023/2023 547/548


In [62]:
selected_stats = {
    'shooting': [
        'Sh',
        'SoT',
        'SoT%',
        'G/Sh',
        'G/SoT',
        'Dist',
        'npxG',
        'npxG/Sh',
        'G-xG',
        'np:G-xG',
    ],
    'passing_types': [
        'Att',
        'Cmp',
        'TB',
        'Sw',
        'Crs',
        'CK'
    ],
    'gca': [
        'SCA'
    ],
    'possession': [
        'Poss',
        'Att',
        'Succ',
        'Succ%'
    ],
    'misc': [
        'CrdY',
        'CrdR',
        'Recov',
    ]
}

In [69]:
def get_value(attr, tds, cols):
    col_index = cols.index(attr)
    return tds[col_index-1].text

def save_game_stats(season, team, opp_team, date, venue, stats, cols, games_dict):
    if venue == 'Home':
        home_team, away_team = team, opp_team
        prefixed_cols = ['home_' + col for col in cols]
    else:
        away_team, home_team = team, opp_team
        prefixed_cols = ['away_' + col for col in cols]
    
    stats_dict = {col: stat for col, stat in zip(prefixed_cols, stats)}
    game_key = (home_team, away_team, date)
    if game_key in games_dict[season]:
        games_dict[season][game_key].update(stats_dict)
    else:
        games_dict[season][game_key] = stats_dict
    
games_stats_dict = {}
    
for season in range(start_season, end_season):
    games_stats_dict[season] = {}
    for squad_idx, si in enumerate(seasons_squad_ids[season]):
        squad_id, team_name = si
        for stat_type in selected_stats.keys():
            clear_output()
            print(f"{season}/{end_season-1} {squad_idx}/{len(seasons_squad_ids[season])} --> {team_name}:{stat_type}")
            season_str = str(season) if single_year_season else f"{season}-{season+1}"
            url = f"https://fbref.com/en/squads/{squad_id}/{season_str}/matchlogs/c{league_id}/{stat_type}"
            driver.get(url)

            rows = driver.find_elements(By.XPATH, '//table/tbody/tr')
            thead = driver.find_elements(By.XPATH, '//table/thead/tr')[1]
            cols = thead.find_elements(By.XPATH, './/child::th')
            cols = [c.text for c in cols]

            for i, r in enumerate(rows):
                if not r.text: continue
                tds = r.find_elements(By.XPATH, './/child::td')
                if not len(tds): continue
                date = r.find_element(By.XPATH, './/child::th').text

                opp_team = get_value('Opponent', tds, cols)
                venue = get_value('Venue', tds, cols)

                stats = []
                for stat_col in selected_stats[stat_type]:
                    stats.append(get_value(stat_col, tds, cols))

                save_game_stats(season, team_name, opp_team, date, venue, stats, selected_stats[stat_type], games_stats_dict)

            time.sleep(6)

2023/2023 28/29 --> Real Salt Lake:misc


In [70]:
driver.close()

In [71]:
def complete_stats(season, game_stats, reg_cols):
    reg_dict = {col: stat for col, stat in zip(reg_cols, game_stats)}
    game_key = (reg_dict['home_team'], reg_dict['away_team'], reg_dict['date'])
    advanced_stats_dict = games_stats_dict[season][game_key]
    game_dict = {**reg_dict, **advanced_stats_dict}
    return game_dict

directory = f"./leagues/{league.lower()}"
if not os.path.exists(directory):
    os.makedirs(directory)

for season in seasons.keys():
    print(f"Total games in the {season} season:", len(seasons[season]))
    
    columns = ['date', 'week', 'home_team', 'home_xg', 'home_score', 'away_score', 'away_xg', 'away_team']
    complete_games = [complete_stats(season, game_stats, columns) for game_stats in seasons[season]]
    
    season_df = pd.DataFrame(complete_games)
    display(season_df)
    season_df.to_csv(f"{directory}/{season}-{season+1}.csv")

Total games in the 2018 season: 408


,date,week,home_team,home_xg,home_score,away_score,away_xg,away_team,home_Sh,home_SoT,...,away_Sw,away_Crs,away_CK,away_SCA,away_Poss,away_Succ,away_Succ%,away_CrdY,away_CrdR,away_Recov
0,2018-03-03,Regular Season,Toronto FC,1.4,0,2,1.5,Columbus Crew,12,2,...,7,15,5,22,42,2,50.0,1,0,72
1,2018-03-03,Regular Season,Dynamo,1.9,4,0,1.8,Atlanta Utd,13,10,...,6,21,4,24,59,4,40.0,3,0,56
2,2018-03-04,Regular Season,Seattle,1.9,0,1,0.3,Los Angeles FC,22,7,...,0,7,1,9,40,12,57.1,1,0,70
3,2018-03-04,Regular Season,Vancouver,1.2,2,1,0.9,Montreal Impact,11,4,...,2,21,4,15,51,15,71.4,2,0,42
4,2018-03-03,Regular Season,Philadelphia,3.5,2,0,0.9,New England,21,6,...,3,12,5,13,37,8,44.4,4,2,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,2018-11-25,Conference Finals,Atlanta Utd,1.1,3,0,0.3,NY Red Bulls,13,5,...,2,10,5,10,49,4,44.4,0,0,50
404,2018-11-25,Conference Finals,Portland Timbers,0.6,0,0,0.6,Sporting KC,14,6,...,1,12,1,15,49,6,42.9,3,0,58
405,2018-11-29,Conference Finals,NY Red Bulls,1.8,1,0,0.5,Atlanta Utd,13,6,...,3,7,4,7,30,6,40.0,0,0,59
406,2018-11-29,Conference Finals,Sporting KC,2.5,2,3,0.8,Portland Timbers,20,8,...,3,8,3,11,41,7,87.5,5,0,37


Total games in the 2019 season: 421


,date,week,home_team,home_xg,home_score,away_score,away_xg,away_team,away_Sh,away_SoT,...,home_Sw,home_Crs,home_CK,home_SCA,home_Poss,home_Succ,home_Succ%,home_CrdY,home_CrdR,home_Recov
0,2019-03-02,Regular Season,Philadelphia,2.8,1,3,1.9,Toronto FC,7,4,...,2,19,8,28,61,6,60.0,2,0,46
1,2019-03-02,Regular Season,Orlando City,1.8,2,2,1.2,NYCFC,13,6,...,8,18,2,24,58,8,47.1,3,0,56
2,2019-03-02,Regular Season,Vancouver,1.1,2,3,2.4,Minnesota Utd,14,3,...,3,16,6,22,49,8,50.0,2,0,58
3,2019-03-02,Regular Season,FC Dallas,0.5,1,1,0.8,New England,9,2,...,7,6,3,14,67,9,52.9,2,0,58
4,2019-03-02,Regular Season,Colorado Rapids,2.2,3,3,2.8,Portland Timbers,23,6,...,5,18,4,24,42,12,80.0,0,1,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,2019-10-24,Conference Semifinals,Los Angeles FC,3.1,5,3,1.7,LA Galaxy,16,6,...,2,4,3,23,43,12,57.1,0,0,48
417,2019-10-24,Conference Semifinals,Atlanta Utd,1.9,2,0,1.0,Philadelphia,12,3,...,4,15,4,22,53,17,73.9,2,0,47
418,2019-10-29,Conference Finals,Los Angeles FC,0.7,1,3,1.2,Seattle,11,5,...,3,18,9,25,68,19,50.0,0,0,64
419,2019-10-30,Conference Finals,Atlanta Utd,2.4,1,2,0.1,Toronto FC,4,4,...,5,22,4,32,59,9,52.9,0,0,56


Total games in the 2020 season: 324


...

,date,week,home_team,home_xg,home_score,away_score,away_xg,away_team,away_Sh,away_SoT,...,home_Sw,home_Crs,home_CK,home_SCA,home_Poss,home_Succ,home_Succ%,home_CrdY,home_CrdR,home_Recov
0,2021-04-17,Regular Season,CF Montréal,1.3,4,2,2.0,Toronto FC,11,2,...,3,9,4,19,50,11,61.1,1,0,45
1,2021-04-17,Regular Season,Los Angeles FC,3.2,2,0,0.8,Austin,12,3,...,1,16,10,40,44,12,50.0,0,0,60
2,2021-04-17,Regular Season,Orlando City,1.4,0,0,1.2,Atlanta Utd,11,3,...,1,8,3,21,41,11,44.0,0,0,48
3,2021-04-16,Regular Season,Seattle,3.2,4,0,1.3,Minnesota Utd,17,4,...,2,14,6,29,50,15,78.9,3,0,49
4,2021-04-16,Regular Season,Dynamo FC,1.7,2,1,1.2,San Jose,8,4,...,0,16,5,29,41,5,35.7,3,0,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,2021-11-28,Conference Semifinals,Philadelphia,2.0,(2) 1,1 (0),0.9,Nashville,14,3,...,3,34,11,38,50,5,55.6,0,0,70
468,2021-11-30,Conference Semifinals,New England,1.1,(3) 2,2 (5),0.9,NYCFC,16,4,...,4,34,4,29,44,4,21.1,2,0,66
469,2021-12-04,Conference Finals,Portland Timbers,1.3,2,0,0.4,Real Salt Lake,7,3,...,3,15,7,25,50,7,41.2,1,0,52
470,2021-12-05,Conference Finals,Philadelphia,0.7,1,2,1.6,NYCFC,11,5,...,1,14,3,13,36,7,50.0,2,0,62


Total games in the 2022 season: 489


,date,week,home_team,home_xg,home_score,away_score,away_xg,away_team,home_Sh,home_SoT,...,away_Sw,away_Crs,away_CK,away_SCA,away_Poss,away_Succ,away_Succ%,away_CrdY,away_CrdR,away_Recov
0,2022-02-26,Regular Season,Los Angeles FC,2.4,3,0,0.4,Colorado Rapids,14,6,...,7,24,5,12,49,6,37.5,1,0,58
1,2022-02-26,Regular Season,Philadelphia,1.1,1,1,1.1,Minnesota Utd,19,5,...,1,13,2,25,47,17,68.0,3,0,71
2,2022-02-26,Regular Season,San Jose,1.1,1,3,3.0,NY Red Bulls,12,2,...,2,8,6,31,31,5,45.5,3,0,70
3,2022-02-26,Regular Season,Columbus Crew,3.4,4,0,0.5,Vancouver,22,7,...,2,16,5,11,43,12,66.7,3,1,49
4,2022-02-26,Regular Season,FC Dallas,1.9,1,1,1.5,Toronto FC,13,3,...,3,13,2,16,56,9,31.0,1,0,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,2022-10-23,Conference Semifinals,CF Montréal,2.6,1,3,1.8,NYCFC,18,8,...,1,1,0,10,37,4,22.2,3,0,44
485,2022-10-23,Conference Semifinals,Austin,1.4,2,1,0.6,FC Dallas,18,5,...,2,17,3,16,53,3,23.1,2,0,36
486,2022-10-30,Conference Finals,Los Angeles FC,2.1,3,0,0.3,Austin,22,9,...,3,9,5,13,49,3,25.0,3,0,50
487,2022-10-30,Conference Finals,Philadelphia,1.7,3,1,0.6,NYCFC,10,6,...,3,9,3,26,64,8,40.0,0,0,66


Total games in the 2023 season: 254


,date,week,home_team,home_xg,home_score,away_score,away_xg,away_team,away_Sh,away_SoT,...,home_Sw,home_Crs,home_CK,home_SCA,home_Poss,home_Succ,home_Succ%,home_CrdY,home_CrdR,home_Recov
0,2023-02-25,Sat,Nashville,1.3,2,0,0.4,NYCFC,9,2,...,2,21,8,16,39,9,60.0,1,0,55
1,2023-02-25,Sat,FC Cincinnati,1.7,2,1,1.4,Dynamo FC,19,7,...,2,5,0,22,38,16,61.5,1,0,63
2,2023-02-25,Sat,FC Dallas,0.9,0,1,0.8,Minnesota Utd,8,4,...,3,27,9,20,62,13,76.5,1,0,48
3,2023-02-25,Sat,Atlanta Utd,1.9,2,1,1.2,San Jose,10,4,...,5,24,8,32,61,16,53.3,2,0,47
4,2023-02-25,Sat,Philadelphia,3.2,4,1,0.6,Columbus Crew,7,2,...,2,10,2,23,42,4,36.4,1,0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2023-06-17,Sat,D.C. United,1.2,1,2,1.8,Real Salt Lake,18,7,...,3,13,4,26,38,9,69.2,3,1,50
250,2023-06-17,Sat,New England,1.0,3,1,0.9,Orlando City,16,4,...,7,16,2,27,59,12,50.0,2,0,67
251,2023-06-17,Sat,Nashville,2.2,3,1,1.6,St. Louis,13,2,...,5,14,3,20,51,9,52.9,4,0,49
252,2023-06-17,Sat,San Jose,1.2,0,0,1.6,Portland Timbers,16,4,...,0,21,7,23,55,12,52.2,3,0,50
